# **This notebook performs ETL tasks**

## Objectives

* This notebook covers 3 main objectives:
  1. Extract data from source.( The raw data is downloaded from Kaggle and stored in the Raw folder)
  2. Transform the data to fit operational needs
  3. Load the data into a new csv file saved in the Cleaned folder.

## Inputs

* Input is the diabetes_prediction_dataset.csv file located in the raw folder(Data main folder).
* Input file is downloaded from Kaggle: https://www.kaggle.com/datasets/iammustafatz/diabetes-prediction-dataset
* The dataset contains contains medical and demographic data of patients along with their diabetes status.
* The 8 features in the dataset include:
  - age                     : Patient's age in years.
  - gender                  : Biological sex of the patient (e.g., Male, Female, Other)
  - body mass index (BMI)   : A measure of body fat based on height and weight (kg/m²).
  - hypertension            : Presence of high blood pressure (1 = Yes, 0 = No).
  - heart disease           : Presence of heart condition (1 = Yes, 0 = No).
  - smoking history         : Patient’s past or current smoking behavior (e.g., never, former, current)
  - HbA1c level             : Average blood sugar level over the past 2-3 months (%).
  - blood glucose level     : Current blood sugar level (usually measured in mg/dL).

* The target variable is:
  - diabetes                : Indicates whether the patient has diabetes (1 = Yes, 0 = No).

## Outputs

* The output is a cleaned CSV file named cleaned_diabetes_data.csv located in the cleaned folder(Data main folder). 

## Additional Comments

* The original dataset is quite huge with 100K records. 
* The target variable is imbalanced with only ~9% positive for diabetes. This could be a representation of real-world prevalence.
* For the purpose of this project, a subset of 10K records is extracted by randomly sampling 5k records from each target class to preserve balance. 
* Subset extraction steps:  
  1. Load the original dataset into a pandas dataframe.
  2. Split the dataset into two for each target class (diabetes = 0 and diabetes = 1).
  3. Randomly sample 5k records from each class.
  4. Concatenate the two sampled datasets to create a balanced subset of 10k records.
  5. Save the sampled records into a new CSV file named sampled_diabetes.csv in the raw folder for the rest of the ETL steps.
  6. The original large dataset is retained in the Raw folder for reference but added to the .gitignore file to avoid commiting large files to the repo.

---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/Healthcare_Diabetes_Analysis/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/Healthcare_Diabetes_Analysis'

# Extraction

* Importing needed libraries

In [4]:
import pandas as pd

* Loading the original raw data into a pandas dataframe

In [5]:
df_original = pd.read_csv('Data/raw/diabetes_prediction_dataset.csv')
df_original.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


* Checking the shape of the original dataframe

In [6]:
df_original.shape

(100000, 9)

* checking basic info of the original dataframe

In [7]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


* checking for class balance in the target variable by displaying value counts as percentage(0 - no diabetes
1 - diabetes)

In [8]:
print("Diabetes value count as %:", df_original['diabetes'].value_counts(normalize=True) * 100)

Diabetes value count as %: diabetes
0    91.5
1     8.5
Name: proportion, dtype: float64


* As we can see from above the dataset is quite huge with 100K records and the target variable is imbalanced with only ~9% positive for diabetes. 
* Extracting only 10k records by randomly sampling 5k records from each target class to preserve balance.

In [9]:
"""
splitting the dataset into 2 by target class and sampling 5000 records from each class to avoid bias
"""

# Separate by class
df_original_no_diab = df_original[df_original['diabetes'] == 0]
df_original_diab = df_original[df_original['diabetes'] == 1]

""" 
Check if each class has at least 5000 records since we wanted 10k records (this check is purely optional)
Setting random state for reproducibility
"""

min_samp_size = 5000
if len(df_original_no_diab) >= min_samp_size and len(df_original_diab) >= min_samp_size:
    # Randomly sample 5000 from each class
    df_no_diab_sampled = df_original_no_diab.sample(n=min_samp_size, random_state=42)
    df_diab_sampled = df_original_diab.sample(n=min_samp_size, random_state=42)

    # Combine and shuffle the final dataset
    df_raw = pd.concat([df_no_diab_sampled, df_diab_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

    print(df_raw.shape)
    print(df_raw['diabetes'].value_counts())
else:
    print("Not enough data to sample 5000 from each class.")

(10000, 9)
diabetes
1    5000
0    5000
Name: count, dtype: int64


* The stats  from the sampled raw dataframe above show 10k records with value counts of the target variable confirming that we have a balanced dataset now.

* Save this sampled raw dataframe to a new csv file in the Raw folder for the rest of the ETL steps

In [10]:
current_dir = os.getcwd()
current_dir

df_raw.to_csv('Data/raw/sampled_diabetes.csv', index=False)


---

# TRANSFORMATION

* In the transformation phase the objective is to have a clean dataset with no missing values, duplicates or invalid data types.
* We do the steps below to achieve this:

  1. Check for missing values and handle them appropriately.
  2. Check for duplicate records and remove them.
  3. Validate data types of each column and convert if necessary.
  4. Perform any additional cleaning steps as needed.

* Further tranasformations like feature engineering, normalization or encoding  will be done later in a feaure engineerting notebook post EDA so that we have a better understanding of the data and do all the appropriate transformations.

In [11]:
# inspectng the first few rows of the dataframe

df_raw.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Male,80.0,0,0,former,26.40,8.2,126,1
1,Female,13.0,0,0,never,30.72,6.1,158,0
2,Male,5.0,0,0,No Info,27.32,6.2,80,0
3,Female,47.0,0,0,never,45.88,4.0,159,0
4,Female,26.0,0,0,not current,27.32,6.0,130,0


In [12]:
# Check the shape of the dataframe
df_raw.shape

(10000, 9)

* Using .info() method, gives a summary of the dataframe including the data types and non-null counts for each column. There by we can check for missing values and inconsistencies in data types of each column.

In [13]:
# Check the data types of the columns and details of the dataframe
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               10000 non-null  object 
 1   age                  10000 non-null  float64
 2   hypertension         10000 non-null  int64  
 3   heart_disease        10000 non-null  int64  
 4   smoking_history      10000 non-null  object 
 5   bmi                  10000 non-null  float64
 6   HbA1c_level          10000 non-null  float64
 7   blood_glucose_level  10000 non-null  int64  
 8   diabetes             10000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 703.3+ KB


* as we can see there are no missing values and all data types are appropriate.
* we can check the age column to see if there are any decimal values in it. If not we can convert it to integer type.
* The column names do not seem to have any spaces, _ used instead which is good for coding purposes. All other column names are lowercase, except for HbA1c which I intend to keep as is since it is a standard medical abbreviation.

#### checking age column for decimal values

In [14]:
# checking for the age if there are any decimal values
df_raw['age'].unique()

array([80.  , 13.  ,  5.  , 47.  , 26.  , 57.  , 63.  , 74.  , 65.  ,
       25.  , 34.  , 60.  , 71.  , 27.  , 35.  , 64.  , 67.  , 75.  ,
       62.  , 59.  , 39.  , 44.  , 43.  , 58.  , 55.  , 70.  , 66.  ,
       53.  , 49.  , 19.  , 22.  , 78.  , 10.  ,  2.  , 20.  , 51.  ,
       76.  , 69.  , 72.  , 77.  , 16.  , 29.  , 23.  , 14.  , 38.  ,
       41.  ,  9.  , 61.  , 54.  , 11.  ,  6.  , 50.  , 37.  , 52.  ,
       17.  , 24.  , 42.  , 73.  , 18.  , 46.  , 36.  , 56.  , 15.  ,
       28.  , 30.  , 45.  ,  7.  , 12.  , 79.  , 40.  , 31.  , 33.  ,
       68.  ,  8.  ,  0.8 , 48.  ,  3.  ,  4.  ,  1.16,  0.64,  1.88,
       32.  , 21.  ,  0.88,  1.48,  1.72,  1.64,  0.32,  0.48,  0.72,
        1.08,  0.08,  1.32,  0.24,  1.56,  0.16,  0.4 ,  0.56,  1.8 ,
        1.24,  1.4 ,  1.  ])

* From the uniquue values of age, there are some infants, toddlers and children. Checking for the count of records which are not adults and decide what to do with them.

In [15]:
# checking the count of records for age below 18 - Non adults

count = (df_raw['age'] < 18).sum()

print(f"Count of non-adults: {count}")    

Count of non-adults: 936


* Since this diabetes analysis dataset also has feature like smoking and bmi which are lifestyle related, and they dont seem of much relevance for infants,toddlers and children below 18 years, I will be removing those records.

In [16]:
# copying the raw dataset to another dataframe before removing the non adults (infants, toddlers and children)

df_raw_copy = df_raw.copy()

# loading the non-adults to a seperate dataframe and checking the shape of the dataframe
df_raw_non_adults = df_raw[df_raw['age'] < 18].copy()
print(df_raw_non_adults.shape)

# loading the adults to a seperate dataframe and checking the shape of the dataframe and value counts of target class
df_raw_adults = df_raw[df_raw['age'] >= 18].copy()
print(df_raw_adults.shape)
print(df_raw_adults['diabetes'].value_counts())

(936, 9)
(9064, 9)
diabetes
1    4955
0    4109
Name: count, dtype: int64


* The value counts on the target variable after removing non adults shows that we have a relatively balanced dataset. So further cleaning steps will be done on this dataframe(df_raw_adults).

In [17]:
# checking for the age if there are any decimal values on the adults dataset
df_raw_adults['age'].unique()

array([80., 47., 26., 57., 63., 74., 65., 25., 34., 60., 71., 27., 35.,
       64., 67., 75., 62., 59., 39., 44., 43., 58., 55., 70., 66., 53.,
       49., 19., 22., 78., 20., 51., 76., 69., 72., 77., 29., 23., 38.,
       41., 61., 54., 50., 37., 52., 24., 42., 73., 18., 46., 36., 56.,
       28., 30., 45., 79., 40., 31., 33., 68., 48., 32., 21.])

* unique values do not show any decimal values. so doing a quick check to confirm if the age column is of integer
 values only and convert the data type to integer  from float.



In [18]:
# check if any of the age column has a fractional part

age_has_fractional = (df_raw_adults['age'] % 1 != 0).any()

if age_has_fractional:
    print(" Age column has fractional values. Cannot convert to integer.")
else:
    print(" Age column has no fractional values. Safe to Convert to integer.")
    

 Age column has no fractional values. Safe to Convert to integer.


In [19]:
# converting age to interger type from float  
df_raw_adults['age'] = df_raw_adults['age'].astype(int)
print("Conversion done.")

Conversion done.


In [20]:
# checking with info to confirm the age column is of integer type
df_raw_adults.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9064 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               9064 non-null   object 
 1   age                  9064 non-null   int64  
 2   hypertension         9064 non-null   int64  
 3   heart_disease        9064 non-null   int64  
 4   smoking_history      9064 non-null   object 
 5   bmi                  9064 non-null   float64
 6   HbA1c_level          9064 non-null   float64
 7   blood_glucose_level  9064 non-null   int64  
 8   diabetes             9064 non-null   int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 708.1+ KB


#### Checking for missing values

In [21]:
df_raw_adults.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

* There are no missing values in any of the columns in the dataframe.

#### checking for duplicates

In [22]:
# checking for count of duplicate values
print("duplicates count before:" , df_raw_adults.duplicated().sum()) 

# checking for all occurances of duplicates 
duplicates = df_raw_adults[df_raw_adults.duplicated(keep=False)]     
duplicates

duplicates count before: 21


,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
531,Male,38,0,0,No Info,31.85,5.7,220,1
691,Male,80,0,0,No Info,27.32,5.7,90,0
952,Male,47,0,0,No Info,27.32,5.8,155,0
954,Female,36,0,0,No Info,27.32,6.6,130,0
1161,Female,22,0,0,No Info,27.32,5.7,126,0
1198,Female,76,0,0,No Info,27.32,6.2,260,1
1303,Female,80,0,0,never,27.32,6.5,160,0
1577,Male,66,0,0,No Info,27.32,5.7,130,1
2107,Female,65,0,0,No Info,27.32,4.0,85,0
2419,Female,47,0,0,No Info,27.32,4.0,100,0


* I have inspected for the duplicate records and they are the exact same records. So I will keep the first occurence and drop the rest.

In [23]:
# removing duplicates by keeping the first occurence and dropping the rest.

df_cleaned = df_raw_adults.drop_duplicates(keep='first').reset_index(drop=True)
df_cleaned.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Male,80,0,0,former,26.40,8.2,126,1
1,Female,47,0,0,never,45.88,4.0,159,0
2,Female,26,0,0,not current,27.32,6.0,130,0
3,Male,80,0,0,No Info,27.32,7.5,160,1
4,Male,57,0,0,No Info,27.32,5.8,158,0


In [24]:
# checking for count of duplicate values post removal of duplicates

print("duplicates count after:" , df_cleaned.duplicated().sum()) 


duplicates count after: 0


#### checking for cardinality and unique values of categorical columns

In [25]:
# checking for cardinality and unique values of the category columns
categorical_columns = ['gender', 'smoking_history']
for col in categorical_columns:
    print(f"{col} cardinality: {df_cleaned[col].nunique()}")
    print(f"{col} unique values:", df_cleaned[col].unique())
    print(f"{col} unique value counts:\n", df_cleaned[col].value_counts())
    print("\n")

gender cardinality: 3
gender unique values: ['Male' 'Female' 'Other']
gender unique value counts:
 gender
Female    5145
Male      3897
Other        1
Name: count, dtype: int64


smoking_history cardinality: 6
smoking_history unique values: ['former' 'never' 'not current' 'No Info' 'ever' 'current']
smoking_history unique value counts:
 smoking_history
never          3535
No Info        2026
former         1352
current         988
not current     685
ever            457
Name: count, dtype: int64




* Since other category in gender is only 1 record, decided to remove that record.
* Will keep the different categories in smoking_history as is for now. Further analysis will be done in the EDA notebook to see if any grouping is needed.

In [26]:
# remove the record with gender - 'Other' since it is only 1 record

df_cleaned = df_cleaned[df_cleaned['gender'] != 'Other'].reset_index(drop=True)
df_cleaned.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Male,80,0,0,former,26.40,8.2,126,1
1,Female,47,0,0,never,45.88,4.0,159,0
2,Female,26,0,0,not current,27.32,6.0,130,0
3,Male,80,0,0,No Info,27.32,7.5,160,1
4,Male,57,0,0,No Info,27.32,5.8,158,0


In [27]:
# checking for the unique values in gender column to confirm removal of 'Other' category

df_cleaned['gender'].unique()

array(['Male', 'Female'], dtype=object)

* Use .describe() method to get statistical summary of numerical columns to check for any anomalies or outliers.

In [28]:
df_cleaned.describe().T

,count,mean,std,min,25%,50%,75%,max
age,9042.0,54.966158,17.211084,18.00,42.0000,57.000,69.0000,80.00
hypertension,9042.0,0.164455,0.370708,0.00,0.0000,0.000,0.0000,1.00
heart_disease,9042.0,0.095665,0.294147,0.00,0.0000,0.000,0.0000,1.00
bmi,9042.0,30.372424,7.090530,10.98,27.0625,27.535,33.4775,88.72
HbA1c_level,9042.0,6.243696,1.295101,3.50,5.7000,6.100,6.8000,9.00
blood_glucose_level,9042.0,166.425680,57.781870,80.00,130.0000,155.000,200.0000,300.00
diabetes,9042.0,0.547003,0.497813,0.00,0.0000,1.000,1.0000,1.00


* The describe() here will be explained in detail in the EDA notebook. We are just checking for a snippet of the data stats.

#### Final check of the cleaned data

In [29]:
# checking the first few records of the cleaned dataframe
df_cleaned.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Male,80,0,0,former,26.40,8.2,126,1
1,Female,47,0,0,never,45.88,4.0,159,0
2,Female,26,0,0,not current,27.32,6.0,130,0
3,Male,80,0,0,No Info,27.32,7.5,160,1
4,Male,57,0,0,No Info,27.32,5.8,158,0


In [30]:
# checking the .info() of the cleaned dataframe to confirm no nulls and correct data types

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9042 entries, 0 to 9041
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               9042 non-null   object 
 1   age                  9042 non-null   int64  
 2   hypertension         9042 non-null   int64  
 3   heart_disease        9042 non-null   int64  
 4   smoking_history      9042 non-null   object 
 5   bmi                  9042 non-null   float64
 6   HbA1c_level          9042 non-null   float64
 7   blood_glucose_level  9042 non-null   int64  
 8   diabetes             9042 non-null   int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 635.9+ KB


In [31]:
# checking for value counts of target class to confirm balance

print(df_cleaned['diabetes'].value_counts())


diabetes
1    4946
0    4096
Name: count, dtype: int64


* With no nulls, no duplicates, no invalid data types and no unwanted records, and relatively balanced target variable -- the dataset is now clean and ready for EDA and further transformations.

# LOAD

---

* In this section we will save/load the cleaned dataframe to cleaned_diabetes_data.csv file located in the cleaned folder(Mian folder - Data).

In [32]:
# loading the cleaned dataframe to new csv file.

current_dir = os.getcwd()
current_dir

df_cleaned.to_csv('Data/cleaned/cleaned_diabetes_data.csv', index=False)

---

# Conclusion and Next Steps

* With the cleaned dataset saved, the next steps will be to perform Exploratory Data Analysis (EDA) to understand the data better and identify patterns, trends, and relationships among the features.
* Plot some visualizations to get insights into the data.
* Perform some statistical testing to validate any hypotheses formed during EDA.